## Discrete Fourier transform

In [3]:
import FFTW
using BenchmarkTools
using PyCall

In [4]:
const x = rand(2 ^ 13);

In [5]:
function dftmatrix(x::AbstractArray)
    n = length(x)
    W = ones(Complex{eltype(x)}, n, n)
    ω = exp(-1im * 2π / n)
    for i = 1:n, j = 1:n
        W[j, i] = ω ^ ((i - 1) * (j - 1))
    end
    W
end

function naivedft(x::AbstractArray)
    dftmatrix(x) * x
end

naivedft (generic function with 1 method)

In [6]:
isapprox(FFTW.fft(x), naivedft(x))

true

In [7]:
function fft_dit(x::AbstractArray)
    n = length(x)
    
    if n == 1
        return x[1]
    end
    
    ω = exp(-2π * im / n)
    
    x_even = fft_dit(x[1:2:end])
    x_odd = fft_dit(x[2:2:end])

    factor = ω .^ collect(0:n-1)
    halfway = n ÷ 2
    vcat(
        x_even .+ factor[1:halfway] .* x_odd,
        x_even .+ factor[halfway+1:end] .* x_odd
    )
end

fft_dit (generic function with 1 method)

In [8]:
isapprox(FFTW.fft(x), fft_dit(x))

true

In [9]:
function bit_reversal!(x::AbstractArray)
    n = length(x)
    j = 1
    for i = 1:n-1
        if i < j
            x[i], x[j] = x[j], x[i]
        end
        k = n ÷ 2
        while k < j
            j -= k
            k ÷= 2
        end
        j += k
    end
end

function fft!(x::AbstractArray{<:Complex})
    n = length(x)
    bit_reversal!(x)
    blocklen = 2
    while blocklen <= n
        middlepoint = blocklen ÷ 2
        for innerindex = 1:middlepoint
            m = -2 * im * (innerindex - 1) / blocklen
            ω = exp(m * π)
            for block = 1:n÷blocklen
                i1 = (block - 1) * blocklen + innerindex
                i2 = i1 + middlepoint
                a = x[i1]
                b = x[i2] * ω
                x[i1] = a + b
                x[i2] = a - b
            end
        end
        blocklen *= 2
    end
end

function fft(x::AbstractArray{<:Real})
    y = complex(copy(x))
    fft!(y)
    y
end

fft (generic function with 1 method)

In [10]:
isapprox(FFTW.fft(x), fft(x))

true

In [17]:
versioninfo()

Julia Version 1.4.2
Commit 44fa15b150* (2020-05-23 18:35 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin17.7.0)
  CPU: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)


In [11]:
@benchmark naivedft(x)

BenchmarkTools.Trial: 
  memory estimate:  1.00 GiB
  allocs estimate:  6
  --------------
  minimum time:     4.578 s (0.01% GC)
  median time:      4.601 s (0.39% GC)
  mean time:        4.601 s (0.39% GC)
  maximum time:     4.624 s (0.77% GC)
  --------------
  samples:          2
  evals/sample:     1

In [12]:
FFTW.set_num_threads(1)
plan = FFTW.plan_fft(x)
@benchmark (plan * x)

BenchmarkTools.Trial: 
  memory estimate:  256.16 KiB
  allocs estimate:  4
  --------------
  minimum time:     49.349 μs (0.00% GC)
  median time:      134.119 μs (0.00% GC)
  mean time:        145.944 μs (10.17% GC)
  maximum time:     3.462 ms (94.28% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [13]:
@benchmark fft_dit(x)

BenchmarkTools.Trial: 
  memory estimate:  15.54 MiB
  allocs estimate:  147470
  --------------
  minimum time:     12.299 ms (0.00% GC)
  median time:      12.520 ms (0.00% GC)
  mean time:        13.259 ms (5.70% GC)
  maximum time:     16.007 ms (14.61% GC)
  --------------
  samples:          377
  evals/sample:     1

In [14]:
@benchmark fft(x)

BenchmarkTools.Trial: 
  memory estimate:  192.16 KiB
  allocs estimate:  4
  --------------
  minimum time:     266.433 μs (0.00% GC)
  median time:      309.516 μs (0.00% GC)
  mean time:        317.771 μs (2.59% GC)
  maximum time:     2.481 ms (87.26% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [15]:
np = pyimport("numpy")
@benchmark np.fft.fft(x)

┌ Warning: both Julia and NumPy are linked with MKL, which may cause conflicts and crashes (#433).
└ @ PyCall /Users/hjkim/.julia/packages/PyCall/zqDXB/src/numpy.jl:73


BenchmarkTools.Trial: 
  memory estimate:  130.00 KiB
  allocs estimate:  42
  --------------
  minimum time:     108.154 μs (0.00% GC)
  median time:      195.517 μs (0.00% GC)
  mean time:        214.026 μs (4.15% GC)
  maximum time:     6.828 ms (44.58% GC)
  --------------
  samples:          10000
  evals/sample:     1